# [실습] Ollama과 vLLM을 이용한 Agentic Work 만들기   



## 라이브러리 설치


이번 실습은 무료 코랩(T4 GPU)이 아닌 고성능 GPU 라이브러리에서 진행합니다.   
무료 코랩으로 진행하시는 경우, GPU 성능의 한계로 vLLM 실행이 어렵습니다.

<br><br>


In [ ]:
!pip install transformers bitsandbytes openai langchain_openai -q

In [ ]:
!pip install langchain_experimental langgraph langchain langchain_community langchain_huggingface dotenv -q

In [ ]:
# Pytorch 버전 호환
# vllm 0.10 : Pytorch 2.7.1
# vllm 0.9.x == 2.7.0
# vllm 0.8.x == 2.6.0
# vllm 0.6.x == 2.5.1

!pip install pyzmq flashinfer-python==0.2.10 vllm==0.10.1 -q
!pip install  flashinfer-python pyzmq vllm -q

설치 후에는 세션을 재시작해 주세요.

In [ ]:
# Flash Attention: 리눅스 전용 설치방법
# Windows 설치는 https://github.com/kingbri1/flash-attention/releases 참고
!pip install flash-attn --no-build-isolation -q


설치할 라이브러리가 많으므로, 가급적 설치 후 세션 재시작을 수행해 주세요.

vLLM은 캐싱을 통해 효과적인 추론과 동시 실행을 지원합니다.   
아래 코드를 터미널에서 실행하세요.

```
vllm serve unsloth/gemma-3-12b-it-bnb-4bit --dtype auto --max_model_len 32768 --quantization bitsandbytes --served_model_name gemma3 --max_num_seqs 1
```

어느 정도 시간이 지나면, vLLM 서빙이 완료됩니다.   
vllm Serve가 정상적으로 완료되면, 8000번 포트에서 모델을 확인하실 수 있습니다.

In [ ]:
# 모델 주소 확인
!curl http://0.0.0.0:8000/v1/models

In [ ]:
from langchain_openai import ChatOpenAI, OpenAI

llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
    model="gemma3",
    temperature=0.5,
    max_tokens=1024
)

In [ ]:
for s in llm.stream("vLLM이 뭐야?"):
    print(s.content, end='')

In [ ]:
def convert_chat(messages, add_generation_prompt = True):
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=add_generation_prompt)

example = [{'role':'user', 'content':'안녕'}]
convert_chat(example)

## 2. HuggingFace LLM과 툴 연동하기

먼저 툴을 설정합니다.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from datetime import datetime

tavily_search = TavilySearchResults(
    max_results=3)

repl_tool = PythonREPLTool()
repl_tool.invoke("for i in range(10): print(i)")


@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    return datetime.now().strftime("%Y-%m-%d")



tools = [tavily_search, repl_tool, current_date]


vLLM에서는 Tool Binding을 수행하려면   
--enable-auto-tool-choice 를 추가한 뒤, 전용 파서를 연결해야 합니다.   
Llama, Qwen, Hermes 등의 모델이 가능합니다.   
https://docs.vllm.ai/en/stable/features/tool_calling.htm

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool


convert_to_openai_tool(tools[2])


툴 설명이 담긴 문자열을 구성합니다.

In [ ]:
tool_desc = str('\n---\n'.join([str(convert_to_openai_tool(tool)) for tool in tools]))
print(tool_desc)

시스템 프롬프트를 구성합니다.   
성능에 매우 중요한 영향을 미치므로, 영어로 작성했습니다.

In [ ]:
system_prompt = f'''
You are a helpful assistant with tools below.
You can decide whether to invoke any functions or not.
If you decide to use any of tools.
print name and required parameters of the tool within a json blob correctly.
For python code, Return the object as a raw dictionary, without escaping quotes or newlines.

for tool use: wrap your output within ```tool_code```.

Example:
```tool_code
{{"name":'name of tool', "arguments":{{List of apparent argument and parameters}}}}
```

When the output of the tool is provided, it will be wrapped within ``tool_output```
Answer accordingly from the result of the tool output.

The question might need some sequential, multiple tool execution.
Think Step by Step.

The following tools are available:
{tool_desc}'''

시스템 프롬프트에 들어가야 하는 내용은 다음과 같습니다.
- Tool Format
- Tool Call Format
- Tool Result Format

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage


messages = [SystemMessage(system_prompt),
            HumanMessage('오늘 날짜가 며칠이니?')]


In [ ]:
response = llm.invoke(messages)
messages.append(response)
response


In [ ]:
print(response.content)

tool_code를 받았으니, 해당 내용을 파싱합니다.

In [ ]:
import ast


def parse_tool(text):
    try:
        text = text.split('```tool_code\n')[1].split('\n```')[0]
        # tool_code로 wrap된 중간 코드 추출

        parsed = ast.literal_eval(text)
        # Dict 형태의 값 변환 (json load와 유사)

        name = parsed.get('name')
        arguments = parsed.get('arguments', {})
        # name과 argument return
        return {'name':name, 'arguments':arguments}
    except (ValueError, SyntaxError):
        return None

result = parse_tool(response.content)
name,arguments = result['name'], result['arguments']
name, arguments

툴 실행을 연결합니다.

In [ ]:
# 툴 이름과 툴 연결
tool_dict = {tool.name: tool for tool in tools}

def execute_tool(name, arguments):
    # 툴 실행한 뒤 tool_output으로 wrap
    result = f'''```tool_output
{tool_dict[name].invoke(arguments)}
```'''
    return result

tool_result = execute_tool(**parse_tool(response.content))

print(tool_result)

In [ ]:
messages.append(HumanMessage(tool_result))
messages[1:]
# 질문 + Tool 요청 + Tool 결과

In [ ]:
# 결과 해석
response = llm.invoke(messages)
response

In [ ]:
messages = [SystemMessage(system_prompt),
            HumanMessage('2025년 4월 발표된 GPT-4.1 모델이 어떤 모델이야? 한국어로 설명해줘.')]
response = llm.invoke(messages)
messages.append(response)
response

In [ ]:
print(response.content)

In [ ]:
tool_result = execute_tool(**parse_tool(response.content))
tool_result

In [ ]:
messages[-1]

In [ ]:
messages.append(HumanMessage(tool_result))
response = llm.invoke(messages)
response

일반적인 입출력 관계의 툴은 이와 같은 방식으로 간단하게 구성할 수 있습니다.   
(만약, Python_REPL과 같이 argument가 복잡한 툴을 수행하는 경우에는   
별도의 함수로 변환하거나 결과물을 수정하는 작업이 필요할 수 있습니다.)

해당 구현을 통해, ReAct Agent 구조를 만들어 보겠습니다.    
bind_tools가 없기 때문에, 기존의 Tool Message를 사용하기 어렵습니다.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages : Annotated[list, add_messages]   # 메시지 맥락을 저장하는 리스트


In [ ]:
from langchain_core.messages import ToolMessage

tool_list = {tool.name: tool for tool in tools}
# tool 목록 dict로 생성

def tool_node(state):
    tool_outputs = []
    tool_call_msgs = state['messages'][-1]
    # 마지막 메시지: 툴 콜링 메시지
    if '```tool_code' in tool_call_msgs.content:
        tool_result = execute_tool(**parse_tool(tool_call_msgs.content))
        # tool 실행 결과 얻기 (결과는 ```tool_output```)
        tool_outputs.append(HumanMessage(tool_result))

    return {'messages': tool_outputs}

def agent(state):
    system_prompt = SystemMessage(f'''
You are a helpful assistant with tools below.
You can decide whether to invoke any functions or not.
If you decide to use any of tools.
print name and required parameters of the tool within a json blob correctly.
For python code, Return the object as a raw dictionary, without escaping quotes or newlines.

for tool use: wrap your output within ```tool_code```.

Example:
```tool_code
{{"name":'name of tool', "arguments":{{List of apparent argument and parameters}}}}
```

When the output of the tool is provided, it will be wrapped within ``tool_output```
Answer accordingly from the result of the tool output.

The question might need some sequential, multiple tool execution.
Think Step by Step.

The following tools are available:
{tool_desc}


Answer in Korean.''')


    response = llm.invoke([system_prompt] + state["messages"])
    return {'messages': response}

def tool_needed(state):

    last_msg = state['messages'][-1]
    if '```tool_code' in last_msg.content: # 툴 콜링이 필요하면
        return "continue"
    else:
        return "finish"

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("agent", agent)
builder.add_node("tools", tool_node)

builder.add_edge(START, 'agent'),
builder.add_conditional_edges("agent",
                              tool_needed,
                               {"continue": "tools","finish": END})
builder.add_edge("tools", "agent")

In [ ]:
graph = builder.compile()
graph

In [ ]:
response = graph.invoke({'messages':[HumanMessage(content="오늘 날짜에 태어난 유명인들 조사해서 알려줘.")]})
response

이번에는 병렬 실행이 가능한 vLLM을 이용해, 리포트 작성 모듈을 구성해 봅시다.

In [ ]:
from typing_extensions import TypedDict, Annotated, Literal, List
from pydantic import BaseModel, Field
from langgraph.graph.message import add_messages

from langchain_core.output_parsers import PydanticOutputParser

# 전체 섹션의 구획: Contents (Chapter List)
# Chapter: name, outline
class Chapter(BaseModel):
    name: str = Field(description="챕터의 이름")
    outline: str = Field(description="챕터의 주요 내용, 1문장 길이로")


class Contents(BaseModel):
    contents: List[Chapter] = Field(description="전체 리포트의 섹션 구성")

parser = PydanticOutputParser(pydantic_object=Contents)

format_str = parser.get_format_instructions()

planner = llm | parser

In [ ]:
import torch

with torch.inference_mode():
    example = planner.invoke(f"LLM의 발전 과정에 대한 보고서 구획을 작성해 주세요. \n{format_str}")
example.contents

In [ ]:
from langchain.output_parsers import OutputFixingParser

# parse 불가능한 출력이 주어지면, llm을 통해 교정하는 파서
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

planner = llm | new_parser

with torch.inference_mode():
    example = planner.invoke(f"LLM의 발전 과정에 대한 3챕터 구성의 보고서 구획을 작성해 주세요. \n{format_str}")
example.contents

그래프에서 사용할 State를 정의합니다.   

이번에는 중간 Writer LLM이 사용할 State를 별도로 만들어 보겠습니다.   
이렇게 구성하면 최종 State에서 필요한 부분만 저장할 수 있습니다.

In [ ]:
import operator

# reducer 구조: operator.add
# 단순 + 연산 구조 (리스트의 + 연산이므로 append)

class State(TypedDict):
    topic: str
    contents: list[Chapter]
    completed_sections: Annotated[list, operator.add]
    final_report: str


# 섹션 Writer가 사용할 State
class SubState(TypedDict):
    chapter: Chapter
    completed_sections: Annotated[list, operator.add]



섹션을 생성하는 노드를 구성합니다.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate

def orchestrator(state: State):

    prompt = ChatPromptTemplate([
        ('system', "주제에 대한 전문가 수준의 깊이 있는 한국어 보고서를 쓰려고 합니다. 보고서의 섹션 구성과, 각 섹션의 간단한 설명을 작성해 주세요."),
        ('user', """주제: {topic}
---

{instruction}

""")
    ])
    chain = prompt.partial(instruction = {format_str}) | planner

    # chain 결과물: Contents (contents: List[Chapter])

    return {"contents": chain.invoke(state).contents}
    # state: topic --> topic
    # Return: List[Chapter]



섹션별 내용을 처리하는 노드를 구성합니다.   
State에는 각각의 Chapter가 아닌 Chapter의 리스트인 Contents가 들어 있는데요.   

`SubState`를 이용해, 각각의 Chapter를 처리하도록 정의하겠습니다.

In [ ]:
def llm_call(state: SubState):
    # SubState :  chapter, completed_sections 2개 property

    chapter = state['chapter']

    prompt = ChatPromptTemplate([
        ('system',"아래 섹션에 대한 상세한 한국어 보고서를 작성하세요." ),
        ('user', "섹션 이름과 주제는 다음과 같습니다: {name} --> {outline}")
    ])

    chain = prompt | llm
    with torch.inference_mode():
        return {"completed_sections": [chain.invoke({'name':chapter.name, 'outline':chapter.outline}).content]}
    # 리스트로 Wrap하는 이유 중요(Reduce Operator 합치기 위해서)


# 생성된 섹션별 결과들을 결합
def synthesizer(state: State):

    completed_sections = state["completed_sections"]

    completed_report_sections = "\n\n---\n\n".join(completed_sections)
    # join: 전체 리스트 스트링으로 결합하기

    return {"final_report": completed_report_sections}




**가장 중요한 부분입니다😁😁**   
langgraph의 Send()를 이용하면, 리스트의 원소 개수만큼 서브모듈을 호출할 수 있습니다.

In [ ]:
from langgraph.constants import Send

def assign_workers(state: State):
    # Send: 노드를 호출하며, 값을 전달해 준다
    # state['contents']의 개수를 기본적으로 알 수 없는데,
    # 이를 통해 개수만큼 llm_call을 생성하여 호출할 수 있음
    with torch.inference_mode():
        return [Send("llm_call", {"chapter": s}) for s in state["contents"]]

그래프를 구성합니다.

In [ ]:
builder = StateGraph(State)

builder.add_node("orchestrator", orchestrator) # 구획 짜고
builder.add_node("llm_call", llm_call) # 섹션별 글쓰고
builder.add_node("synthesizer", synthesizer) # 합치고


builder.add_edge(START, "orchestrator")

builder.add_conditional_edges("orchestrator", assign_workers, ["llm_call"])
# assign_workers의 결과에 따라 llm_call을 호출

builder.add_edge("llm_call", "synthesizer")
# 생성된 섹션들은 synthesizer로 이동

builder.add_edge("synthesizer", END) # 끝


graph = builder.compile()
# graph

In [ ]:
with torch.inference_mode():
    for data in graph.stream({"topic": "GPT 1부터 최신 LLM까지의 발전과정 (총 3챕터 길이)"}, stream_mode='updates'):
        print(data)
        print('--------------')
        # 생성은 병렬적이지만 합치는 순서는 호출한 순서

In [ ]:
from IPython.display import Markdown
print(data['synthesizer']["final_report"])